In [3]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from pymongo import MongoClient
from dotenv import load_dotenv

In [4]:
# Nama database dan collection
db_name = 'youtube_rewind_indonesia'
collection_name = 'labeling_2015'

# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

# Membuat koneksi ke MongoDB
client = MongoClient(local_url)

db = client[db_name]
collection = db[collection_name]

In [6]:
# Mengambil data dari MongoDB ke dalam dataframe
data = list(collection.find())
df = pd.DataFrame(data)

# Memisahkan data yang berlabel (2.000) dan tidak berlabel (8.000)
df_labeled = df.iloc[:2000]  # 2.000 data yang berlabel
df_unlabeled = df.iloc[2000:]  # 8.000 data yang tidak berlabel

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# Melatih model menggunakan Naive Bayes
model = MultinomialNB()
model.fit(X_labeled_tfidf, y_labeled)

# Memproses data yang tidak berlabel
X_unlabeled = df_unlabeled['textOriginal']
X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# Memprediksi label untuk data yang tidak berlabel
y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# Menambahkan prediksi ke dalam dataframe data tidak berlabel
df_unlabeled['predicted_sentiment'] = y_unlabeled_pred

# Menyimpan hasil prediksi ke dalam file CSV
df_unlabeled.to_csv('unlabeled_data_with_predictions.csv', index=False)

print("Prediksi label untuk data tidak berlabel telah selesai.")


Prediksi label untuk data tidak berlabel telah selesai.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3532\2335506333.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlabeled['predicted_sentiment'] = y_unlabeled_pred
